In [ ]:
Добавлено:
1) one-hot
2) обработка пропущенных значений - mean вместо nan
3) вывод элементов выборки
4) обновлён params
TODO:
1) подбор k
2) попробовать большое по модулю негативное число место nan

In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import scipy.linalg as sc
from sklearn import preprocessing
from utils import qualityMACAPE, qualityRMSE
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm_notebook
from sklearn.utils.extmath import softmax
%matplotlib inline
np.random.seed(68)

In [2]:
# softmax([[1.,3,3],[3.,1.,0] ])
# >>> array([[0.06337894, 0.46831053, 0.46831053],
# >>> [0.84379473, 0.1141952 , 0.04201007]])

# datetime.strptime('17DEC2018','%d%b%Y').toordinal()
# >>> 737045

In [3]:
prod_feat = ['STORE_LOCATION_LVL_NM2', 'ST_LOC_DIVISION_NM', 'ST_LOC_MNGMT_CLSTR', 'PRODUCT_DESC']

In [4]:
ts = pd.read_csv('./EGG_train.csv', encoding="cp1251", sep=';', decimal=',', parse_dates=True, dayfirst=True)
# parse dates
ts['period_start_dt'] = [ datetime.strptime(date, '%d%b%Y').toordinal() for date in ts['period_start_dt'] ]
ts = ts.sort_values(by=['PRODUCT_ID', 'STORE_LOCATION_ID', 'period_start_dt'])
ts = ts.groupby(['PRODUCT_ID', 'STORE_LOCATION_ID']).agg(
    lambda x: list(x)[0] if type(list(x)[0]) is str else list(x) )

In [5]:
ts.iloc[:2]

period_start_dt  \
PRODUCT_ID STORE_LOCATION_ID                                                      
475861     833805             [737045, 737046, 737047, 737048, 737049, 73705...   
           833807             [737045, 737046, 737047, 737048, 737049, 73705...   

                                                                     demand_qty  \
PRODUCT_ID STORE_LOCATION_ID                                                      
475861     833805             [5, 15, 3, 3, 4, 2, 2, 4, 4, 7, 13, 13, 13, 11...   
           833807             [40, 50, 36, 34, 55, 38, 35, 49, 36, 60, 58, 5...   

                             STORE_LOCATION_LVL_NM2 STORE_LOCATION_LVL_NM7  \
PRODUCT_ID STORE_LOCATION_ID                                                 
475861     833805                  59 ПЕРМСКИЙ КРАЙ    22_1119160 ЛЫСЬВА Г   
           833807                  72 ТЮМЕНСКАЯ ОБЛ    22_1180983 ТЮМЕНЬ Г   

                             ST_LOC_DIVISION_NM ST_LOC_MNGMT_CLSTR  \
PRODUCT_ID STORE_LOCATION_ID                                         
475861     833805                            UR             D_UR17   
           833807                            UR             D_UR29   

                               PRODUCT_LVL_NM8             PRODUCT_LVL_NM13  \
PRODUCT_ID STORE_LOCATION_ID                                                  
475861     833805             FR0607000 - Яйцо  42_FR0607001 - Яйцо куриное   
           833807             FR0607000 - Яйцо  42_FR0607001 - Яйцо куриное   

                                                  PRODUCT_DESC  
PRODUCT_ID STORE_LOCATION_ID                                    
475861     833805             ТЕНДЕР Яйца кур.С0 стол.фас.10шт  
           833807             ТЕНДЕР Яйца кур.С0 стол.фас.10шт

In [6]:
base_algs = pd.read_csv('./EGG_BA_FCST.csv', encoding="cp1251", sep=';',
                        decimal=',', parse_dates=True, dayfirst=True)
# parse dates
base_algs['dt'] = [ datetime.strptime(date, '%d%b%Y').toordinal() for date in base_algs['dt'] ]
base_algs = base_algs.sort_values(by=['PRODUCT_LVL_ID', 'STORE_LOCATION_LVL_ID', 'dt']).groupby(
    ['PRODUCT_LVL_ID', 'STORE_LOCATION_LVL_ID']).agg(lambda x: list(x))

In [7]:
base_algs.iloc[:2]

FC_WK  \
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                      
475861         833805                 [17DEC2018, 17DEC2018, 17DEC2018, 17DEC2018, 1...   
               833807                 [17DEC2018, 17DEC2018, 17DEC2018, 17DEC2018, 1...   

                                                                                     dt  \
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                      
475861         833805                 [737045, 737046, 737047, 737048, 737049, 73705...   
               833807                 [737045, 737046, 737047, 737048, 737049, 73705...   

                                                                                REGULAR  \
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                      
475861         833805                 [13.808, 14.767, 14.151, 14.225, 16.6, 14.393,...   
               833807                 [48.201, 47.105, 48.23, 45.698, 48.673, 45.587...   

                                                                                SAS_FAW  \
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                      
475861         833805                 [11.402, 14.406, 13.243, 12.417, 15.854, 15.62...   
               833807                 [46.673, 46.091, 49.205, 44.513, 51.553, 49.42...   

                                                                              SAS_MINER  \
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                      
475861         833805                 [14.043, 13.101, 13.042, 13.885, 15.022, 11.39...   
               833807                 [43.755, 42.337, 41.579, 41.251, 40.291, 36.73...   

                                                                           Horizon_Week  \
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                      
475861         833805                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
               833807                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                                                            Horizon_day  
PRODUCT_LVL_ID STORE_LOCATION_LVL_ID                                                     
475861         833805                 [1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 1, ...  
               833807                 [1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 1, ...

In [12]:
def arbitrating_composition(ts_df, base_algs, h, params ):
    ''' Parameters:
    ts <Pandas DataFrame> - data frame of time series with features (=prod_feat)
    base_algs <Pandas DataFrame> - data frame of base algorithms answers
    h <integer scalar> - forecasting delay
    params <dict>
        product_features - features of product (don't include demand, date)
        base_algs_num - number of base algorithms
        training_step - train every <training_step>'s time stamps
        k - size of sample for meta-learner
    '''
    prod_feat = params['product_features']
    base_algs_num = params['base_algs_num']
    step = params['training_step']
    k = params['k']
    
    ts = ts_df.copy()
    min_date = min(list(map(min, ts['period_start_dt'])))
    forecast = np.empty( (ts.shape[0], max(list(map(max, ts['period_start_dt']))) - min_date + h) )
    forecast[:] = np.nan

    for pf in prod_feat:
        ts[pf] = preprocessing.LabelEncoder().fit_transform(ts[pf])
    oh = [preprocessing.OneHotEncoder().fit( ts[pf].as_matrix().reshape(-1, 1) ) for pf in prod_feat]

    X_train, Y_train = None, None
    rf = [ RandomForestRegressor() for _ in range(base_algs_num) ]  # random forest
    is_fitted = False
    
    print(f'Wait {len(ts)} iterations')
    for ts_n, ((pr_id, st_id), _) in tqdm_notebook(enumerate(ts.iterrows()) ):
        ba = base_algs.loc[pr_id, st_id]
        forecast_ba = np.empty( (base_algs_num, forecast.shape[1]), dtype=np.float )
        forecast_ba[:] = np.nan
        for i, date in enumerate( ba['dt'] ):
            if date - min_date >= forecast_ba.shape[1]:
                break
            forecast_ba[0, date - min_date] = ba['REGULAR'][i]
            forecast_ba[1, date - min_date] = ba['SAS_FAW'][i]
            forecast_ba[2, date - min_date] = ba['SAS_MINER'][i]
        time_ser = np.empty( forecast.shape[1], dtype=np.float )
        time_ser[:] = np.nan
        ts_x = ts.loc[pr_id, st_id]
        for i, date in enumerate( ts_x['period_start_dt'] ):
            time_ser[date - min_date] = ts_x['demand_qty'][i]
        # mean value instead of nan
        time_ser[np.isnan( time_ser )] = time_ser[np.logical_not( np.isnan( time_ser ) )].mean()
        
        xx = sc.hankel( time_ser[:-(k-1)], [-1]+list(time_ser[-(k-1):]) )[:-h, :]
        xx_f = np.hstack( [ (oh[j].transform([[ ts.loc[pr_id, st_id][prod_feat[j]] ]]) ).toarray()[0] \
                          for j in range(len(prod_feat)) ] )
        xx = np.hstack( [xx, np.tile( xx_f.reshape(-1), (xx.shape[0], 1) ) ] )

        yy = np.vstack([ time_ser[k-1+h:] for i in range(base_algs_num) ])
        yy = np.abs(forecast_ba[:, k-1+h: k-1+h + yy.shape[1]] - yy )
        #nn_ind = np.logical_not(np.isnan( yy.sum(axis = 0) ))
        yy_step_c = 0
        for t in range( yy.shape[1] + k-1+h ):
            if (t >= k-1+h) and is_fitted:
                x_t = xx[t - (k-1+h)]
                ans = [ rf[i].predict([x_t])[0] for i in range(len(rf)) ]
                forecast[ts_n, t] = np.dot(softmax([ans])[0], forecast_ba[:, t])
            if not np.isnan(yy[:, t-(k-1+h)]).any():
                yy_step_c += 1
            if yy_step_c % step == 0:
                #x = np.array([xx[i, :] for i in range(len(nn_ind)) if nn_ind[i]])
                #y = np.array([yy[:, i] for i in range(len(nn_ind)) if nn_ind[i]]).T
                x, y = xx[:], yy[:]
                for rf_i in range(len(rf)):
                    if X_train is None:
                        batch_x, batch_y = x[:t-(k-1+h)], y[rf_i, :t-(k-1+h)]
                    else:
                        batch_x = np.vstack( (X_train, x[:t-(k-1+h)] ))
                        batch_y = np.append(Y_train[rf_i, :], y[rf_i, :t-(k-1+h)])
                    if batch_y.shape[0] != 0:
                        rf[rf_i] = RandomForestRegressor()
                        rf[rf_i].fit( batch_x, batch_y )
                        is_fitted = True
        if X_train is None:
            X_train, Y_train = xx, yy
        else:
            X_train, Y_train = np.vstack( (X_train, xx) ), np.hstack( (Y_train, yy) )
    return forecast

In [10]:
params = {'product_features': prod_feat, 'training_step': 7, 'k': 8, 'base_algs_num': 3}

In [14]:
#forecast = arbitrating_composition(ts, base_algs, 1, params)

In [58]:
print('Количество уникальных значений для prod_feat:')
for f in prod_feat:
    print(f,  np.unique(ts[f]).shape[0] )

Количество уникальных значений для prod_feat:
STORE_LOCATION_LVL_NM2 64
STORE_LOCATION_LVL_NM7 1917
ST_LOC_DIVISION_NM 15
ST_LOC_MNGMT_CLSTR 198
PRODUCT_LVL_NM8 1
PRODUCT_LVL_NM13 1
PRODUCT_DESC 3


In [62]:
print('Три значения вр:')
print(ts.iloc[9747])
#ts[np.logical_and( (ts['PRODUCT_ID'] == 475861),ts['STORE_LOCATION_ID'] == 845840)]

Три значения вр:
period_start_dt                   [737046, 737049, 737051]
demand_qty                                               1
STORE_LOCATION_LVL_NM2                    63 САМАРСКАЯ ОБЛ
STORE_LOCATION_LVL_NM7               22_1081621 ТОЛЬЯТТИ Г
ST_LOC_DIVISION_NM                                      VO
ST_LOC_MNGMT_CLSTR                                  D_VO53
PRODUCT_LVL_NM8                           FR0607000 - Яйцо
PRODUCT_LVL_NM13               42_FR0607001 - Яйцо куриное
PRODUCT_DESC              ТЕНДЕР Яйца кур.С0 стол.фас.10шт
Name: (475861, 845840), dtype: object


In [27]:
# forecast[ np.logical_not(np.isnan(forecast)) ].shape

# for i, col in enumerate(ts.columns):
#     ts[col].plot(figsize=(15,5), color='green', label='real')
#     pd.Series(data=forecast[i, :-1], index=ts[col].index).plot(color='red', label='arbitrated')
    
#     plt.title(col)
#     plt.legend()
#     plt.show()